In [100]:
from glob import glob
import matplotlib.pyplot as plt
import gc
import xarray as xr
import numpy as np
import sys
sys.path.append("/homes/metogra/iufarias/Documents/Scripts/useful")
import romspickle
import seawater as sw
import pandas as pd
import matplotlib.gridspec as gridspec

import dask
dask.config.set(**{'array.slicing.split_large_chunks': True})

plt.rcParams['font.size']=15
plt.rcParams['font.sans-serif']='Arial'
plt.rcParams['font.weight']='bold'

In [2]:

# chunk={'time_counter':20}
# path='/data/pacific/lrenault/SASI/FULL/WRF/'
# listm=glob(path+'*DAY.nc');listm.sort()
# # varf=xr.open_dataset(listm[-9:][xmon])
# var=xr.open_mfdataset(listm[7:],chunks=chunk)


chunk={'time_counter':20}
path='/data/pacific/lrenault/SASI/FULL/WRF/'
listw=glob(path+'wrfout*.nc');listw.sort()
varw=xr.open_mfdataset(listw[7:],chunks=chunk)



In [48]:
u_w=varw.U
v_w=varw.V

uw = 0.5 * (u_w[:,:,:,:-1] + u_w[:,:,:,1:])
vw = 0.5 * (v_w[:,:,:-1,:] + v_w[:,:,1:,:])


# uw=uw.rename({'y_grid_U':'y_grid','x_grid_U':'x_grid'})
# vw=vw.rename({'y_grid_V':'y_grid','x_grid_V':'x_grid'})




In [43]:
speed=np.sqrt(uw**2 + vw**2)

In [50]:
u_w

<xarray.DataArray 'U' (time_counter: 213, lev_M: 49, y_grid_U: 390, x_grid_U: 301)>
dask.array<concatenate, shape=(213, 49, 390, 301), dtype=float32, chunksize=(20, 49, 390, 301), chunktype=numpy.ndarray>
Coordinates:
    nav_lat_grid_U  (y_grid_U, x_grid_U) float32 dask.array<chunksize=(390, 301), meta=np.ndarray>
    nav_lon_grid_U  (y_grid_U, x_grid_U) float32 dask.array<chunksize=(390, 301), meta=np.ndarray>
  * lev_M           (lev_M) float32 1.0 2.0 3.0 4.0 5.0 ... 46.0 47.0 48.0 49.0
    time_centered   (time_counter) datetime64[ns] dask.array<chunksize=(20,), meta=np.ndarray>
  * time_counter    (time_counter) datetime64[ns] 2012-01-01T12:00:00 ... 201...
Dimensions without coordinates: y_grid_U, x_grid_U
Attributes:
    long_name:           x-wind component
    units:               m s-1
    online_operation:    average
    interval_operation:  12 s
    interval_write:      1 d
    cell_methods:        time: mean (interval: 12 s)

In [248]:
id=0
plt.figure(figsize=(10,8))
(varw.SST[id]-273).plot(x='nav_lon_grid_M',y='nav_lat_grid_M',levels=np.linspace(10,15,35),cmap='Spectral_r')
plt.plot(varw.nav_lon_grid_M[-60,:],varw.nav_lat_grid_M[-60,:],'k--')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title(pd.DatetimeIndex(varw.time_centered)[id])
plt.tight_layout()
plt.savefig('/homes/metogra/iufarias/Documents/figures/1_WRF/map_LLJ_{0}.png'.format(pd.DatetimeIndex(varw.time_centered)[id]))
plt.close()

In [210]:
tt0

<xarray.DataArray 'T2' ()>
dask.array<mean_agg-aggregate, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    time_centered  datetime64[ns] dask.array<chunksize=(), meta=np.ndarray>
    time_counter   datetime64[ns] 2012-01-01T12:00:00

In [222]:
(tt0.data/(-6.5e-3))*(1-(pbb/pbb.data[0])**((287*(-6.5e-3))/9.8))



<xarray.DataArray (lev_M: 13)>
dask.array<mul, shape=(13,), dtype=float32, chunksize=(13,), chunktype=numpy.ndarray>
Coordinates:
  * lev_M          (lev_M) float32 1.0 2.0 3.0 4.0 5.0 ... 10.0 11.0 12.0 13.0
    time_centered  datetime64[ns] dask.array<chunksize=(), meta=np.ndarray>
    time_counter   datetime64[ns] 2012-01-01T12:00:00

In [223]:
def press2height(p,p0,t0,gamma=-6.5e-3,R=287.053,g=9.8):
    alt=(t0/gamma)*(1-(p/p0)**((R*gamma)/g))
    return alt


In [240]:
pbb=varw.PB[0,0:13,-60,:200].mean(axis=1)

In [249]:
for id in range(varw.SST.shape[0]):
    
    tt0=varw.T2[0,-60,:200].mean()
    hh=press2height(p=pbb,p0=pbb[0].data,t0=tt0.data)
    
    pc=plt.figure(figsize=(8, 12 ))

    gs = gridspec.GridSpec(100,10)
    ax1 = plt.subplot(gs[:75, :])

    cc=ax1.contourf(varw.SST.nav_lon_grid_M[-60,:200],hh,speed[id,0:13,-60,:200],
                levels=np.linspace(0,20,35),cmap='Spectral_r',extend='max')
    plt.colorbar(cc)
    # ax1.set_ylim(1000,850)
    ax1.set_xlim(varw.SST.nav_lon_grid_M[-60,:200].min(),varw.SST.nav_lon_grid_M[-60,:200].max())
    ax1.set_xticklabels('')
    ax1.set_title('Windspeed [$m\,s^{-1}$]')

    ax2 = plt.subplot(gs[80:, 0:8])

    plt.plot(varw.SST[id,-60,:200].nav_lon_grid_M,varw.SST[id,-60,:200]-273)
    ax2.set_xlim(varw.SST.nav_lon_grid_M[-60,:200].min(),varw.SST.nav_lon_grid_M[-60,:200].max())
    ax2.set_ylabel(r'$SST \,\,(^{\circ}C)$')
    ax2.set_ylim(9,13)
    ax2.set_title(pd.DatetimeIndex(varw.time_centered)[id])

    plt.tight_layout()
    plt.savefig('/homes/metogra/iufarias/Documents/figures/1_WRF/LLJ/section_{0}.png'.format(pd.DatetimeIndex(varw.time_centered)[id]))
    plt.close()


/homes/metogra/iufarias/miniconda3/envs/ROMS/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
/homes/metogra/iufarias/miniconda3/envs/ROMS/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
/homes/metogra/iufarias/miniconda3/envs/ROMS/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
/homes/metogra/iufarias/miniconda3/envs/ROMS/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
/homes/metogra/iufarias/miniconda3/envs/ROMS/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserW